<a href="https://colab.research.google.com/github/OlajideFemi/GeneralContent/blob/master/MailMerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gspread google-auth google-api-python-client jinja2


In [3]:
# 1. Setup credentials and authorize access to Google Sheets, Drive, and Gmail
sheet = gspread.open_by_key(1VFsnz3_4dOcHq7Q0JklwwP2hQwLw_Sb8dm7xfikEqSE).worksheet("Sheet1")
rows = sheet.get_all_records()

for row_idx, row in enumerate(rows, start=2):  # assuming row 1 = headers
    if row["Email Sent"]:
        continue

    recipient = row["Recipient"]
    file_id = row["Attachment Sheet ID"]

    # 2. Download the attachment sheet as PDF
    file_data = download_sheet_as_pdf(file_id)

    # 3. Get Gmail draft and merge placeholders (e.g. {{First Name}})
    subject, body = render_gmail_draft_with_row(row)

    # 4. Send the email via Gmail API with attachment
    send_email_with_attachment(recipient, subject, body, file_data)

    # 5. Update the "Email Sent" column
    sheet.update_cell(row_idx, sent_col_index, timestamp_now())


SyntaxError: invalid decimal literal (ipython-input-3-3582268046.py, line 2)

In [4]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account

from googleapiclient.discovery import build
import base64, os, io
from email.message import EmailMessage
import xlsxwriter

# Authorize access to Google Sheets
#gc = gspread.authorize(auth.get_user_credentials())


In [ ]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build
import base64, os, io
from email.message import EmailMessage
import xlsxwriter
from datetime import datetime

# Initialize services
gc = gspread.authorize(auth.get_user_credentials())
service = build('gmail', 'v1')

def get_sheet_data(sheet_url):
    """Get data from Google Sheet"""
    sh = gc.open_by_url(sheet_url)
    worksheet = sh.sheet1
    data = worksheet.get_all_records()
    headers = worksheet.row_values(1)
    return data, headers

def create_excel_attachment(data, filename):
    """Create Excel attachment from data"""
    output = io.BytesIO()
    workbook = xlsxwriter.Workbook(output)
    worksheet = workbook.add_worksheet()

    # Write headers
    headers = list(data[0].keys())
    for col, header in enumerate(headers):
        worksheet.write(0, col, header)

    # Write data
    for row, item in enumerate(data, start=1):
        for col, key in enumerate(headers):
            worksheet.write(row, col, str(item.get(key, '')))

    workbook.close()
    output.seek(0)
    return output.getvalue()

def send_email(recipient, subject, message_text, attachment_data=None, attachment_name='data.xlsx'):
    """Send email with optional attachment"""
    message = EmailMessage()
    message['To'] = recipient
    message['Subject'] = subject
    message.set_content(message_text)

    if attachment_data:
        message.add_attachment(attachment_data, maintype='application',
                             subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                             filename=attachment_name)

    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {'raw': encoded_message}

    try:
        message = service.users().messages().send(userId='me', body=body).execute()
        return message
    except Exception as e:
        print(f'Error sending email to {recipient}: {str(e)}')
        return None

def run_mail_merge(sheet_url, subject_template, message_template):
    """Main mail merge function"""
    data, headers = get_sheet_data(sheet_url)

    for row in data:
        if not row.get('Email Sent', ''):  # Only send if not already sent
            # Customize message
            subject = subject_template.format(**row)
            message = message_template.format(**row)

            # Create attachment
            attachment_data = create_excel_attachment([row], f'data_{row["Recipient"]}.xlsx')

            # Send email
            result = send_email(
                recipient=row['Recipient'],
                subject=subject,
                message_text=message,
                attachment_data=attachment_data
            )

            if result:
                print(f"Email sent to {row['Recipient']}")
                # Mark as sent in the sheet (would need write permissions)
                # worksheet.update_cell(row_index, email_sent_col, str(datetime.now()))
            else:
                print(f"Failed to send to {row['Recipient']}")

# Example usage
SHEET_URL = "YOUR_SHEET_URL_HERE"
SUBJECT_TEMPLATE = "Your report for {Name}"
MESSAGE_TEMPLATE = """Hello {Name},

Here's your personalized report.

Regards,
The Team"""

run_mail_merge(SHEET_URL, SUBJECT_TEMPLATE, MESSAGE_TEMPLATE)

In [5]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from googleapiclient.discovery import build
import base64, os, io
from email.message import EmailMessage
import xlsxwriter
from datetime import datetime

# Initialize services
gc = gspread.authorize(auth.get_user_credentials())
service = build('gmail', 'v1')

def get_sheet_data(sheet_url):
    """Get data from Google Sheet"""
    sh = gc.open_by_url(sheet_url)
    worksheet = sh.sheet1
    data = worksheet.get_all_records()
    headers = worksheet.row_values(1)
    return data, headers

def create_excel_attachment(data, filename):
    """Create Excel attachment from data"""
    output = io.BytesIO()
    workbook = xlsxwriter.Workbook(output)
    worksheet = workbook.add_worksheet()

    # Write headers
    headers = list(data[0].keys())
    for col, header in enumerate(headers):
        worksheet.write(0, col, header)

    # Write data
    for row, item in enumerate(data, start=1):
        for col, key in enumerate(headers):
            worksheet.write(row, col, str(item.get(key, '')))

    workbook.close()
    output.seek(0)
    return output.getvalue()

def send_email(recipient, subject, message_text, attachment_data=None, attachment_name='data.xlsx'):
    """Send email with optional attachment"""
    message = EmailMessage()
    message['To'] = recipient
    message['Subject'] = subject
    message.set_content(message_text)

    if attachment_data:
        message.add_attachment(attachment_data, maintype='application',
                             subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                             filename=attachment_name)

    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    body = {'raw': encoded_message}

    try:
        message = service.users().messages().send(userId='me', body=body).execute()
        return message
    except Exception as e:
        print(f'Error sending email to {recipient}: {str(e)}')
        return None

def run_mail_merge(sheet_url, subject_template, message_template):
    """Main mail merge function"""
    data, headers = get_sheet_data(sheet_url)

    for row in data:
        if not row.get('Email Sent', ''):  # Only send if not already sent
            # Customize message
            subject = subject_template.format(**row)
            message = message_template.format(**row)

            # Create attachment
            attachment_data = create_excel_attachment([row], f'data_{row["Recipient"]}.xlsx')

            # Send email
            result = send_email(
                recipient=row['Recipient'],
                subject=subject,
                message_text=message,
                attachment_data=attachment_data
            )

            if result:
                print(f"Email sent to {row['Recipient']}")
                # Mark as sent in the sheet (would need write permissions)
                # worksheet.update_cell(row_index, email_sent_col, str(datetime.now()))
            else:
                print(f"Failed to send to {row['Recipient']}")

# Example usage
SHEET_URL = "YOUR_SHEET_URL_HERE"
SUBJECT_TEMPLATE = "Your report for {Name}"
MESSAGE_TEMPLATE = """Hello {Name},

Here's your personalized report.

Regards,
The Team"""

run_mail_merge(SHEET_URL, SUBJECT_TEMPLATE, MESSAGE_TEMPLATE)

AttributeError: module 'google.colab.auth' has no attribute 'get_user_credentials'

In [ ]:
/**
 * @OnlyCurrentDoc
 */

/**
 * Change these to match the column names you are using for email
 * recipient addresses and email sent column.
 */
const RECIPIENT_COL = "Recipient";
const EMAIL_SENT_COL = "Email Sent";
// NEW: Add a constant for the column containing the Google Sheet ID for the attachment
const ATTACHMENT_SHEET_ID_COL = "Attachment Sheet ID";

/** * Creates the menu item "Mail Merge" for user to run scripts on drop-down.
 */
function onOpen() {
  const ui = SpreadsheetApp.getUi();
  ui.createMenu('Mail Merge')
      .addItem('Send Emails', 'sendEmails')
      .addToUi();
}

/**
 * Sends emails from sheet data.
 * @param {string} subjectLine (optional) for the email draft message
 * @param {Sheet} sheet to read data from
 */
function sendEmails(subjectLine, sheet = SpreadsheetApp.getActiveSheet()) {
  // option to skip browser prompt if you want to use this code in other projects
  if (!subjectLine) {
    subjectLine = Browser.inputBox("Mail Merge",
                                   "Type or copy/paste the subject line of the Gmail " +
                                   "draft message you would like to mail merge with:",
                                   Browser.Buttons.OK_CANCEL);

    if (subjectLine === "cancel" || subjectLine == "") {
      // If no subject line, finishes up
      return;
    }
  }

  // Gets the draft Gmail message to use as a template
  const emailTemplate = getGmailTemplateFromDrafts_(subjectLine);

  // Gets the data from the passed sheet
  const dataRange = sheet.getDataRange();
  const data = dataRange.getDisplayValues();

  // Assumes row 1 contains our column headings
  const heads = data.shift();

  // Gets the index of the column named 'Email Status' (Assumes header names are unique)
  const emailSentColIdx = heads.indexOf(EMAIL_SENT_COL);
  // NEW: Get index for Attachment Sheet ID column
  const attachmentSheetIdColIdx = heads.indexOf(ATTACHMENT_SHEET_ID_COL);

  // Converts 2d array into an object array
  const obj = data.map(r => (heads.reduce((o, k, i) => (o[k] = r[i] || '', o), {})));

  // Creates an array to record sent emails
  const out = [];

  // Loops through all the rows of data
  obj.forEach(function(row, rowIdx) {
    // Only sends emails if email_sent cell is blank and not hidden by a filter
    if (row[EMAIL_SENT_COL] == '') {
      try {
        const msgObj = fillInTemplateFromObject_(emailTemplate.message, row);

        // --- NEW ATTACHMENT LOGIC START ---
        let attachmentBlob = null;
        const recipientSheetId = row[ATTACHMENT_SHEET_ID_COL];

        if (recipientSheetId) {
          try {
            const spreadsheetFile = DriveApp.getFileById(recipientSheetId);
            // Convert the Google Sheet to PDF.
            // You can also use MimeType.MICROSOFT_EXCEL to export as .xlsx
            attachmentBlob = spreadsheetFile.getAs(MimeType.PDF)
                                            .setName(`${row[RECIPIENT_COL].split('@')[0]}_Report.pdf`); // Customize filename
          } catch (e) {
            Logger.log(`Error getting or converting sheet ID ${recipientSheetId}: ${e.message}`);
            // If there's an issue with the attachment, log it and proceed without it
            // or you might choose to throw an error and stop for this recipient.
          }
        }
        // --- NEW ATTACHMENT LOGIC END ---

        const emailOptions = {
          htmlBody: msgObj.html,
          attachments: emailTemplate.attachments, // Existing attachments from draft
          inlineImages: emailTemplate.inlineImages
        };

        // Add the dynamically generated attachment if it exists
        if (attachmentBlob) {
          if (!emailOptions.attachments) {
            emailOptions.attachments = [];
          }
          emailOptions.attachments.push(attachmentBlob);
        }

        GmailApp.sendEmail(row[RECIPIENT_COL], msgObj.subject, msgObj.text, emailOptions);

        // Edits cell to record email sent date
        out.push([new Date()]);
      } catch (e) {
        // modify cell to record error
        out.push([e.message]);
      }
    } else {
      out.push([row[EMAIL_SENT_COL]]);
    }
  });

  // Updates the sheet with new data
  sheet.getRange(2, emailSentColIdx + 1, out.length).setValues(out);

  /**
   * Get a Gmail draft message by matching the subject line.
   * @param {string} subject_line to search for draft message
   * @return {object} containing the subject, plain and html message body and attachments
   */
  function getGmailTemplateFromDrafts_(subject_line) {
    try {
      // get drafts
      const drafts = GmailApp.getDrafts();
      // filter the drafts that match subject line
      const draft = drafts.filter(subjectFilter_(subject_line))[0];
      // get the message object
      const msg = draft.getMessage();

      // Handles inline images and attachments so they can be included in the merge
      // Based on https://stackoverflow.com/a/65813881/1027723
      // Gets all attachments and inline image attachments
      const allInlineImages = draft.getMessage().getAttachments({includeInlineImages: true, includeAttachments: false});
      const attachments = draft.getMessage().getAttachments({includeInlineImages: false});
      const htmlBody = msg.getBody();

      // Creates an inline image object with the image name as key
      // (can't rely on image index as array based on insert order)
      const img_obj = allInlineImages.reduce((obj, i) => (obj[i.getName()] = i, obj), {});

      //Regexp searches for all img string positions with cid
      const imgexp = RegExp('<img.*?src="cid:(.*?)".*?alt="(.*?)"[^>]+>', 'g');
      const matches = [...htmlBody.matchAll(imgexp)];

      //Initiates the allInlineImages object
      const inlineImagesObj = {};
      // built an inlineImagesObj from inline image matches
      matches.forEach(match => inlineImagesObj[match[1]] = img_obj[match[2]]);

      return {message: {subject: subject_line, text: msg.getPlainBody(), html: htmlBody},
        attachments: attachments, inlineImages: inlineImagesObj};
    } catch (e) {
      throw new Error("Oops - can't find Gmail draft. Error: " + e.message);
    }

    /**
     * Filter draft objects with the matching subject linemessage by matching the subject line.
     * @param {string} subject_line to search for draft message
     * @return {object} GmailDraft object
     */
    function subjectFilter_(subject_line) {
      return function(element) {
        if (element.getMessage().getSubject() === subject_line) {
          return element;
        }
      };
    }
  }

  /**
   * Fill template string with data object
   * @see https://stackoverflow.com/a/378000/1027723
   * @param {object} template object containing {{}} markers which are replaced with data
   * @param {object} data object used to replace {{}} markers
   * @return {object} message replaced with data
   */
  function fillInTemplateFromObject_(template, data) {
    // We have two templates one for plain text and the html body
    // Stringifing the object means we can do a global replace
    let template_string = JSON.stringify(template);

    // Token replacement
    template_string = template_string.replace(/{{[^{}]+}}/g, key => {
      return escapeData_(data[key.replace(/[{}]+/g, "")] || "");
    });
    return JSON.parse(template_string);
  }

  /**
   * Escape cell data to make JSON safe
   * @see https://stackoverflow.com/a/9204218/1027723
   * @param {string} str to escape JSON special characters from
   * @return {string} escaped string
   */
  function escapeData_(str) {
    return str
        .replace(/[\\]/g, '\\\\')
        .replace(/[\"]/g, '\\\"')
        .replace(/[\/]/g, '\\/')
        .replace(/[\b]/g, '\\b')
        .replace(/[\f]/g, '\\f')
        .replace(/[\n]/g, '\\n')
        .replace(/[\r]/g, '\\r')
        .replace(/[\t]/g, '\\t');
  }
}


https://docs.google.com/spreadsheets/d/1VFsnz3_4dOcHq7Q0JklwwP2hQwLw_Sb8dm7xfikEqSE/edit?gid=0#gid=0

https://docs.google.com/spreadsheets/d/156-V-oVqOEAdKEJJ0vLydp4Vdr5aC2QeOpZA9buejd4/edit?gid=0#gid=0



In [6]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter
from google.colab import auth
auth.authenticate_user()

In [8]:
SHEET_URL = ""  # Paste your Google Sheet URL
SUBJECT_TEMPLATE = "Your report for {Name}"  # Use {ColumnName} for placeholders
MESSAGE_TEMPLATE = """Hello {Name}, ..."""

In [9]:
#hhgh

In [10]:
!pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread xlsxwriter pandas
from google.colab import auth
auth.authenticate_user()

import gspread
import pandas as pd
from googleapiclient.discovery import build
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import base64, io, time
from datetime import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.


In [11]:
gc = gspread.authorize(auth.get_user_credentials())
service = build('gmail', 'v1')

# Load your sheet
sheet_url = "YOUR_SHEET_URL"  # e.g., "https://docs.google.com/spreadsheets/d/123abc/edit"
sh = gc.open_by_url(sheet_url)
worksheet = sh.sheet1
data = worksheet.get_all_records()  # Gets data as a list of dictionaries

In [ ]:
def create_advanced_excel(data, recipient_email):
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        # Convert to DataFrame for easy processing
        df = pd.DataFrame(data)

        # Sheet 1: Raw Data
        df.to_excel(writer, sheet_name='Report', index=False)

        # Sheet 2: Summary (with formulas)
        workbook = writer.book
        worksheet = workbook.add_worksheet('Summary')

        # Add formats
        header_format = workbook.add_format({'bold': True, 'bg_color': '#4F81BD', 'font_color': 'white'})

        # Write data with formatting
        worksheet.write_row(0, 0, ['Metric', 'Value'], header_format)
        worksheet.write(1, 0, 'Total Records')
        worksheet.write(1, 1, f'=COUNT(Report!A:A)-1')  # Excel formula

        # Add a chart
        chart = workbook.add_chart({'type': 'column'})
        chart.add_series({'values': '=Report!B1:B10'})
        worksheet.insert_chart('D1', chart)

    output.seek(0)
    return output.getvalue()

In [ ]:
def send_email(recipient, subject, html_body, excel_data=None):
    message = MIMEMultipart()
    message['to'] = recipient
    message['subject'] = subject

    # HTML Email Body (with dynamic placeholders)
    message.attach(MIMEText(html_body, 'html'))

    # Attach Excel
    if excel_data:
        excel_attachment = MIMEApplication(excel_data, _subtype="xlsx")
        excel_attachment.add_header(
            'Content-Disposition',
            'attachment',
            filename=f'report_{datetime.now().date()}.xlsx'
        )
        message.attach(excel_attachment)

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    service.users().messages().send(userId='me', body={'raw': raw}).execute()

In [ ]:
for row in data:
    if not row.get('Email Sent'):
        try:
            # Customize content per recipient
            html_content = f"""
            <html>
                <body>
                    <p>Hello <strong>{row['Name']}</strong>,</p>
                    <p>Your custom report is attached.</p>
                </body>
            </html>
            """

            # Generate advanced Excel
            excel_data = create_advanced_excel([row], row['Email'])

            # Send email
            send_email(
                recipient=row['Email'],
                subject=f"Advanced Report for {row['Name']}",
                html_body=html_content,
                excel_data=excel_data
            )

            # Mark as sent
            worksheet.update_cell(row['row_index'], email_sent_col_idx, str(datetime.now()))
            print(f"Sent to {row['Email']}")

        except Exception as e:
            print(f"Failed for {row['Email']}: {str(e)}")

        time.sleep(1)  # Avoid rate limits

In [ ]:
if row['Status'] == 'VIP':
    send_vip_email(row)
else:
    send_standard_email(row)

In [ ]:
!gdown --id YOUR_GOOGLE_DRIVE_FILE_ID
with open('template.html') as f:
    html_template = f.read()

In [ ]:
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(send_email, recipients)

In [ ]:
log_sheet = sh.add_worksheet(title="Logs", rows=100, cols=4)
log_sheet.append_row(["Timestamp", "Recipient", "Status", "Error"])